# NLTK Corpus Analysis with Gensim's LDA Model 

## Preparation
First of all, you need to import necessary libraries (with pip command).
* nltk
* gensim
* pyLDAvis

In [197]:
!pip install nltk
!pip install gensim
!pip install pyLDAvis

After installing the dependencies, you need to download the following datasets.

In [198]:
import nltk
nltk.download("stopwords")
nltk.download("wordnet")
nltk.download("semcor")
nltk.download("punkt")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Shireen\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Shireen\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package semcor to
[nltk_data]     C:\Users\Shireen\AppData\Roaming\nltk_data...
[nltk_data]   Package semcor is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Shireen\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

## Datasets
Load the corpus from NLTK package.

In [199]:
from nltk.corpus import semcor as corpus

Let us check out the content of the corpus.

In [200]:
for n,item in enumerate(corpus.words(corpus.fileids()[0])[:100000]):
    print(item, end=" ")
    if (n%25) ==24:
      print(" ")
    
print("")
print("")
print("Total number of documents:",len(corpus.fileids()))

The Fulton County Grand Jury said Friday an investigation of Atlanta 's recent primary election produced `` no evidence '' that any irregularities took place  
. The jury further said in term end presentments that the City Executive Committee , which had over-all charge of the election , `` deserves  
the praise and thanks of the City of Atlanta '' for the manner in which the election was conducted . The September October term jury  
had been charged by Fulton Superior Court Judge Durwood Pye to investigate reports of possible `` irregularities '' in the hard-fought primary which was won  
by Mayor-nominate Ivan Allen Jr. . `` Only a relative handful of such reports was received '' , the jury said , `` considering the  
widespread interest in the election , the number of voters and the size of this city '' . The jury said it did find that  
many of Georgia 's registration and election laws `` are outmoded or inadequate and often ambiguous '' . It recommended that Fulton legislators act

You can train the model with first K number of documents or all documents.

In [201]:
# All documents
docs=[corpus.words(fileid) for fileid in corpus.fileids()]

print(docs[:10])
print ("")
print("num of docs:", len(docs))

[['The', 'Fulton', 'County', 'Grand', 'Jury', 'said', ...], ['Committee', 'approval', 'of', 'Gov.', 'Price', ...], ['The', 'Orioles', 'tonight', 'retained', 'the', ...], ['A', 'Texas', 'halfback', 'who', 'does', "n't", ...], ['Rookie', 'Ron', 'Nischwitz', 'continued', 'his', ...], ['Nick', 'Skorich', ',', 'the', 'line', 'coach', 'for', ...], ['If', 'the', 'Cardinals', 'heed', 'Manager', 'Gene', ...], ['Sizzling', 'temperatures', 'and', 'hot', 'summer', ...], ['The', 'nuclear', 'war', 'is', 'already', 'being', ...], ['It', 'is', 'not', 'news', 'that', 'Nathan', ...]]

num of docs: 352


## Data preprocessing
First, let us define some stopwords. Here we consider English stopwords from the NLTK package and some noises that may affect our LDA analysis result.  
(Optional) Try to ignore numbers and words through regular expression.

In [202]:
# English stopwords defined by the NLTK package.
from nltk.corpus import stopwords
en_stop = stopwords.words("english")

# Ignore noises that might affect our result.
en_stop = ["``","/",",.",".,",";","--",":",")","(",'"','&',"'",'),',',"','-','.,','.,"','.-',"?",">","<","$","''","*","!"]\
+["0","1","2","3","4","5","6","7","8","9","10","87","50","100","31","29","13","637","71","30","74","1119","18","1913","1923","1937","1961","1962","87","402"]\
+["f","a's" ,"mr.", "able" , "about" , "above" , "according" , "accordingly" , "across" , "actually" , "after" , "afterwards" , "again" , "against" , "ain't" , "all" , "allow" , "allows" , "almost" , "alone" , "along" , "already" , "also" , "although" , "always" , "am" , "among" , "amongst" , "an" , "and" , "another" , "any" , "anybody" , "anyhow" , "anyone" , "anything" , "anyway" , "anyways" , "anywhere" , "apart" , "appear" , "appreciate" , "appropriate" , "are" , "aren't" , "around" , "as" , "aside" , "ask" , "asking" , "associated" , "at" , "available" , "away" , "awfully" , "be" , "became" , "because" , "become" , "becomes" , "becoming" , "been" , "before" , "beforehand" , "behind" , "being" , "believe" , "below" , "beside" , "besides" , "best" , "better" , "between" , "beyond" , "both" , "brief" , "but" , "by" , "c'mon" , "c's" , "came" , "can" , "can't" , "cannot" , "cant" , "cause" , "causes" , "certain" , "certainly" , "changes" , "clearly" , "co" , "com" , "come" , "comes" , "concerning" , "consequently" , "consider" , "considering" , "contain" , "containing" , "contains" , "corresponding" , "could" , "couldn't" , "course" , "currently" , "definitely" , "described" , "despite" , "did" , "didn't" , "different" , "do" , "does" , "doesn't" , "doing" , "don't" , "done" , "down" , "downwards" , "during" , "each" , "edu" , "eg" , "eight" , "either" , "else" , "elsewhere" , "enough" , "entirely" , "especially" , "et" , "etc" , "even" , "ever" , "every" , "everybody" , "everyone" , "everything" , "everywhere" , "ex" , "exactly" , "example" , "except" , "far" , "few" , "fifth" , "first" , "five" , "followed" , "following" , "follows" , "for" , "former" , "formerly" , "forth" , "four" , "from" , "further" , "furthermore" , "get" , "gets" , "getting" , "given" , "gives" , "go" , "goes" , "going" , "gone" , "got" , "gotten" , "greetings" , "had" , "hadn't" , "happens" , "hardly" , "has" , "hasn't" , "have" , "haven't" , "having" , "he" , "he's" , "hello" , "help" , "hence" , "her" , "here" , "here's" , "hereafter" , "hereby" , "herein" , "hereupon" , "hers" , "herself" , "hi" , "him" , "himself" , "his" , "hither" , "hopefully" , "how" , "howbeit" , "however" , "i'd" , "i'll" , "i'm" , "i've" , "ie" , "if" , "ignored" , "immediate" , "in" , "inasmuch" , "inc" , "indeed" , "indicate" , "indicated" , "indicates" , "inner" , "insofar" , "instead" , "into" , "inward" , "is" , "isn't" , "it" , "it'd" , "it'll" , "it's" , "its" , "itself" , "just" , "keep" , "keeps" , "kept" , "know" , "known" , "knows" , "last" , "lately" , "later" , "latter" , "latterly" , "least" , "less" , "lest" , "let" , "let's" , "like" , "liked" , "likely" , "little" , "look" , "looking" , "looks" , "ltd" , "mainly" , "many" , "may" , "maybe" , "me" , "mean" , "meanwhile" , "merely" , "might" , "more" , "moreover" , "most" , "mostly" , "much" , "must" , "my" , "myself" , "name" , "namely" , "nd" , "near" , "nearly" , "necessary" , "need" , "needs" , "neither" , "never" , "nevertheless" , "new" , "next" , "nine" , "no" , "nobody" , "non" , "none" , "noone" , "nor" , "normally" , "not" , "nothing" , "novel" , "now" , "nowhere" , "obviously" , "of" , "off" , "often" , "oh" , "ok" , "okay" , "old" , "on" , "once" , "one" , "ones" , "only" , "onto" , "or" , "other" , "others" , "otherwise" , "ought" , "our" , "ours" , "ourselves" , "out" , "outside" , "over" , "overall" , "own" , "particular" , "particularly" , "per" , "perhaps" , "placed" , "please" , "plus" , "possible" , "presumably" , "probably" , "provides" , "que" , "quite" , "qv" , "rather" , "rd" , "re" , "really" , "reasonably" , "regarding" , "regardless" , "regards" , "relatively" , "respectively" , "right" , "said" , "same" , "saw" , "say" , "saying" , "says" , "second" , "secondly" , "see" , "seeing" , "seem" , "seemed" , "seeming" , "seems" , "seen" , "self" , "selves" , "sensible" , "sent" , "serious" , "seriously" , "seven" , "several" , "shall" , "she" , "should" , "shouldn't" , "since" , "six" , "so" , "some" , "somebody" , "somehow" , "someone" , "something" , "sometime" , "sometimes" , "somewhat" , "somewhere" , "soon" , "sorry" , "specified" , "specify" , "specifying" , "still" , "sub" , "such" , "sup" , "sure" , "t's" , "take" , "taken" , "tell" , "tends" , "th" , "than" , "thank" , "thanks" , "thanx" , "that" , "that's" , "thats" , "the" , "their" , "theirs" , "them" , "themselves" , "then" , "thence" , "there" , "there's" , "thereafter" , "thereby" , "therefore" , "therein" , "theres" , "thereupon" , "these" , "they" , "they'd" , "they'll" , "they're" , "they've" , "think" , "third" , "this" , "thorough" , "thoroughly" , "those" , "though" , "three" , "through" , "throughout" , "thru" , "thus" , "to" , "together" , "too" , "took" , "toward" , "towards" , "tried" , "tries" , "truly" , "try" , "trying" , "twice" , "two" , "un" , "under" , "unfortunately" , "unless" , "unlikely" , "until" , "unto" , "up" , "upon" , "us" , "use" , "used" , "useful" , "uses" , "using" , "usually" , "value" , "various" , "very" , "via" , "viz" , "vs" , "want" , "wants" , "was" , "wasn't" , "way" , "we" , "we'd" , "we'll" , "we're" , "we've" , "welcome" , "well" , "went" , "were" , "weren't" , "what" , "what's" , "whatever" , "when" , "whence" , "whenever" , "where" , "where's" , "whereafter" , "whereas" , "whereby" , "wherein" , "whereupon" , "wherever" , "whether" , "which" , "while" , "whither" , "who" , "who's" , "whoever" , "whole" , "whom" , "whose" , "why" , "will" , "willing" , "wish" , "with" , "within" , "without" , "won't" , "wonder" , "would" , "wouldn't" , "yes" , "yet" , "you" , "you'd" , "you'll" , "you're" , "you've" , "your" , "yours" , "yourself" , "yourselves" , "zero"]\
+["sunday","friday","september","october","million","said","tuesday","wednesday","friday","years","monday","november","jan.","aug.","sept.","go","back","one","year"]\
+[" of ","gov.","w.","add","size","jr.","end","mrs.","m.","see","rep.","b.","d.","13th","yet","n't","want","put","p.","make","say","let","tabb","gop","tom","four"]\
+en_stop

Next, let us define several preprocessing functions.

In [203]:
from nltk.corpus import wordnet as wn # import for lemmatize

def preprocess_word(word, stopwordset):
    
    #1.convert words to lowercase (e.g., Python =>python)
    word=word.lower()
    
    #2.remove ",", ".", and "'s"
    if word in [",",".","'s"]:
        return None
    
    #3.remove stopwords  (e.g., the => (None)) 
    if word in stopwordset:
        return None
    
    #4.lemmatize  (e.g., cooked=>cook)
    lemma = wn.morphy(word)
    if lemma is None:
        return word

    # lemmatized words could be in the stopwords set
    elif lemma in stopwordset: 
        return None
    else:
        return lemma
    

def preprocess_document(document):
    document=[preprocess_word(w, en_stop) for w in document]
    document=[w for w in document if w is not None]
    return document

def preprocess_documents(documents):
    return [preprocess_document(document) for document in documents]

Let us check out the preprocessing result.

In [204]:
# before
print(docs[0][:10]) 

# after
print(preprocess_documents(docs)[0][:10])

['The', 'Fulton', 'County', 'Grand', 'Jury', 'said', 'Friday', 'an', 'investigation', 'of']
['fulton', 'county', 'grand', 'jury', 'investigation', 'atlanta', 'recent', 'primary', 'election', 'produce']


Next, we need to reshape our documents with the available format for the gensim LDA model.

In [205]:
import gensim
from gensim import corpora

In [206]:
# build the dictionary
dictionary = corpora.Dictionary(preprocess_documents(docs))
# construct the corpus
corpus_ = [dictionary.doc2bow(doc) for doc in preprocess_documents(docs)]

Let us check out the contents of the built dictionary and corpus.

In [207]:
# token2id is the attribute which indicates the mapping between words and dictionary ID

print(dictionary.token2id)

{'1958': 0, 'accept': 1, 'achieve': 2, 'act': 3, 'action': 4, 'additional': 5, 'adjournment': 6, 'adjustment': 7, 'administration': 8, 'administrator': 9, 'afternoon': 10, 'age': 11, 'agree': 12, 'agriculture': 13, 'aid': 14, 'airport': 15, 'ala.': 16, 'allen': 17, 'allot': 18, 'allowance': 19, 'alpharetta': 20, 'alternative': 21, 'ambiguous': 22, 'amendment': 23, 'amicable': 24, 'announce': 25, 'anonymous': 26, 'apparently': 27, 'appoint': 28, 'appointment': 29, 'appraiser': 30, 'approve': 31, 'area': 32, 'arm': 33, 'assistance': 34, 'assistant': 35, 'association': 36, 'atlanta': 37, 'attend': 38, 'attorney': 39, 'audience': 40, 'authorities': 41, 'authority': 42, 'automobile': 43, 'awarding': 44, 'ballot': 45, 'bar': 46, 'barber': 47, 'battle': 48, 'begin': 49, 'belief': 50, 'bellwood': 51, 'berry': 52, 'birth': 53, 'bit': 54, 'blue': 55, 'board': 56, 'body': 57, 'bond': 58, 'bowden': 59, 'bring': 60, 'burden': 61, 'bush': 62, 'byrd': 63, 'caldwell': 64, 'call': 65, 'callan': 66, 'ca

In [208]:
# corpus_ contains words of each document with a list (ID, appear frequency)

# note that there is not the appearing order in the documents, but the order of the dictionary
print(corpus_[0][:10]) 

[(0, 1), (1, 1), (2, 1), (3, 2), (4, 4), (5, 2), (6, 2), (7, 1), (8, 2), (9, 1)]


Let us compare the original document with our preprocessing result that is available for the LDA model.

In [209]:
# before
print([w.lower() for w in corpus.sents(corpus.fileids()[0])[0]])

# after
print(dictionary.doc2bow([w.lower() for w in corpus.sents(corpus.fileids()[0])[0]]))

['the', 'fulton', 'county', 'grand', 'jury', 'said', 'friday', 'an', 'investigation', 'of', 'atlanta', "'s", 'recent', 'primary', 'election', 'produced', '``', 'no', 'evidence', "''", 'that', 'any', 'irregularities', 'took', 'place', '.']
[(37, 1), (108, 1), (151, 1), (165, 1), (196, 1), (211, 1), (249, 1), (262, 1), (340, 1), (360, 1), (392, 1)]


## Training with k = 10

In [224]:
ldamodel = gensim.models.ldamodel.LdaModel(corpus=corpus_,
                                           num_topics=10,
                                           id2word=dictionary,
                                           alpha=0.1,                 # optional LDA hyperparameter alpha
                                           eta=0.1,                  # optional LDA hyperparameter beta
                                           minimum_probability=0.1    # optional the lower bound of the topic/word generative probability
                                          )

Check out the learned parameters.

In [225]:
# the top num_words of words for each topic (topic ID, the word generative probability for the topic).

topics = ldamodel.print_topics(num_words=10)
for topic in topics:
    print(topic)

(0, '0.004*"state" + 0.003*"time" + 0.002*"man" + 0.002*"world" + 0.002*"work" + 0.002*"house" + 0.002*"people" + 0.002*"school" + 0.002*"interest" + 0.002*"life"')
(1, '0.004*"state" + 0.003*"time" + 0.002*"people" + 0.002*"man" + 0.002*"american" + 0.002*"house" + 0.002*"show" + 0.002*"life" + 0.002*"high" + 0.002*"long"')
(2, '0.004*"man" + 0.004*"state" + 0.004*"time" + 0.002*"day" + 0.002*"work" + 0.002*"good" + 0.002*"life" + 0.002*"school" + 0.002*"place" + 0.002*"long"')
(3, '0.003*"time" + 0.003*"man" + 0.002*"point" + 0.002*"state" + 0.002*"line" + 0.002*"good" + 0.002*"order" + 0.002*"give" + 0.002*"life" + 0.002*"general"')
(4, '0.003*"state" + 0.003*"time" + 0.002*"school" + 0.002*"world" + 0.002*"man" + 0.002*"area" + 0.002*"work" + 0.002*"good" + 0.002*"people" + 0.002*"high"')
(5, '0.003*"time" + 0.002*"man" + 0.002*"state" + 0.002*"people" + 0.002*"long" + 0.002*"life" + 0.002*"work" + 0.002*"american" + 0.002*"country" + 0.002*"turn"')
(6, '0.005*"time" + 0.003*"man" 

In [226]:
# for each document, show the probabilities of topics which beyond the minimum_probability [(topic ID, probability)]

for n,item in enumerate(corpus_[:10]):
    print("document ID "+str(n)+":" ,end="")
    print(ldamodel.get_document_topics(item))

document ID 0:[(0, 0.6145426), (4, 0.23849651), (5, 0.10943411)]
document ID 1:[(0, 0.10345514), (2, 0.7528706)]
document ID 2:[(3, 0.9991144)]
document ID 3:[(2, 0.38252598), (3, 0.61663693)]
document ID 4:[(2, 0.5438167), (3, 0.17743744), (4, 0.277988)]
document ID 5:[(0, 0.11199094), (3, 0.88718593)]
document ID 6:[(2, 0.5567761), (6, 0.4045134)]
document ID 7:[(2, 0.41048336), (6, 0.5132844)]
document ID 8:[(0, 0.34666404), (4, 0.6082511)]
document ID 9:[(1, 0.21787263), (3, 0.6924529)]


Let us check out the ```nth``` document in the result.

In [227]:
n=0

# nth document's topic distribution
print(ldamodel.get_document_topics(corpus_[n]))

# nth document's category
#print(categories[n])

# show the original document
print(" ".join(docs[n]))

[(0, 0.6128956), (4, 0.2459963)]
The Fulton County Grand Jury said Friday an investigation of Atlanta 's recent primary election produced `` no evidence '' that any irregularities took place . The jury further said in term end presentments that the City Executive Committee , which had over-all charge of the election , `` deserves the praise and thanks of the City of Atlanta '' for the manner in which the election was conducted . The September October term jury had been charged by Fulton Superior Court Judge Durwood Pye to investigate reports of possible `` irregularities '' in the hard-fought primary which was won by Mayor-nominate Ivan Allen Jr. . `` Only a relative handful of such reports was received '' , the jury said , `` considering the widespread interest in the election , the number of voters and the size of this city '' . The jury said it did find that many of Georgia 's registration and election laws `` are outmoded or inadequate and often ambiguous '' . It recommended that F

## Visualization
We can further analyze our result through visualization.

In [228]:
import pyLDAvis.gensim
pyLDAvis.enable_notebook()
lda_display = pyLDAvis.gensim.prepare(ldamodel, corpus_, dictionary)
lda_display

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
8      0.005951 -0.004728       1        1  15.189014
7      0.006943  0.004495       2        1  12.859397
9     -0.006167  0.000762       3        1  12.742886
2     -0.004057  0.002096       4        1  12.595822
6     -0.002855  0.013270       5        1   9.988265
5      0.000528 -0.001487       6        1   9.791417
0      0.001404 -0.006395       7        1   8.305712
1      0.005505 -0.000424       8        1   7.149373
3     -0.011903 -0.005764       9        1   6.043640
4      0.004651 -0.001826      10        1   5.334468, topic_info=         Term         Freq        Total Category  logprob  loglift
459     state  1077.000000  1077.000000  Default  30.0000  30.0000
481      time  1023.000000  1023.000000  Default  29.0000  29.0000
32       area   375.000000   375.000000  Default  28.0000  28.0000
202   general   307.000000   307.000000  Default  27.0000  27.0000
433    school   447.000000   447.000000  Default  26.0000  26.0000
...       ...          ...          ...      ...      ...      ...
206      give    22.946835   425.472687  Topic10  -6.5730   0.0110
65       call    22.259539   421.032593  Topic10  -6.6034  -0.0090
279      long    23.943329   523.832520  Topic10  -6.5305  -0.1545
3358     life    21.363581   517.096924  Topic10  -6.6445  -0.2556
1667      men    20.861042   478.652924  Topic10  -6.6683  -0.2021

[835 rows x 6 columns], token_table=      Topic      Freq Term
term                      
7666      1  0.130336    %
7666      2  0.243294    %
7666      3  0.069513    %
7666      4  0.112958    %
7666      5  0.069513    %
...     ...       ...  ...
8399      7  0.061079    |
8399      8  0.061079    |
8399      9  0.244315    |
9740      2  0.301015   |a
9740      7  0.301015   |a

[3706 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[9, 8, 10, 3, 7, 6, 1, 2, 4, 5])

## Training with k = 20

In [229]:
ldamodel = gensim.models.ldamodel.LdaModel(corpus=corpus_,
                                           num_topics=20,
                                           id2word=dictionary,
                                           alpha=0.1,                 # optional LDA hyperparameter alpha
                                           eta=0.1,                   # optional LDA hyperparameter beta
                                           minimum_probability=0.1    # optional the lower bound of the topic/word generative probability
                                          )

Check out the learned parameters.

In [230]:
# the top num_words of words for each topic (topic ID, the word generative probability for the topic).

topics = ldamodel.print_topics(num_words=10)
for topic in topics:
    print(topic)

(0, '0.004*"state" + 0.003*"time" + 0.002*"good" + 0.002*"day" + 0.002*"house" + 0.002*"world" + 0.002*"high" + 0.002*"work" + 0.002*"american" + 0.002*"church"')
(1, '0.005*"state" + 0.003*"time" + 0.003*"school" + 0.002*"man" + 0.002*"place" + 0.002*"program" + 0.002*"work" + 0.002*"child" + 0.002*"tax" + 0.002*"city"')
(2, '0.003*"man" + 0.003*"state" + 0.002*"world" + 0.002*"time" + 0.002*"people" + 0.002*"work" + 0.002*"life" + 0.002*"american" + 0.002*"long" + 0.001*"men"')
(3, '0.003*"time" + 0.003*"man" + 0.002*"state" + 0.002*"world" + 0.002*"work" + 0.002*"good" + 0.002*"call" + 0.002*"life" + 0.002*"place" + 0.002*"house"')
(4, '0.003*"time" + 0.003*"state" + 0.003*"man" + 0.002*"god" + 0.002*"show" + 0.002*"work" + 0.002*"life" + 0.002*"day" + 0.002*"good" + 0.002*"people"')
(5, '0.003*"time" + 0.002*"man" + 0.002*"state" + 0.002*"show" + 0.002*"american" + 0.002*"head" + 0.002*"life" + 0.002*"day" + 0.001*"world" + 0.001*"give"')
(6, '0.004*"time" + 0.003*"state" + 0.002*"

In [231]:
# for each document, show the probabilities of topics which beyond the minimum_probability [(topic ID, probability)]

for n,item in enumerate(corpus_[:10]):
    print("document ID "+str(n)+":" ,end="")
    print(ldamodel.get_document_topics(item))

document ID 0:[(1, 0.29576778), (16, 0.6856898)]
document ID 1:[(0, 0.30170026), (1, 0.6965632)]
document ID 2:[(5, 0.5208367), (8, 0.1084774), (11, 0.3020786)]
document ID 3:[(11, 0.28753704), (18, 0.41458726), (19, 0.2628872)]
document ID 4:[(12, 0.99794334)]
document ID 5:[(0, 0.4429041), (12, 0.39879638)]
document ID 6:[(7, 0.7835198), (12, 0.21448433)]
document ID 7:[(12, 0.12077444), (13, 0.42124537), (17, 0.4189019)]
document ID 8:[(11, 0.9978531)]
document ID 9:[(2, 0.40846476), (17, 0.58740866)]


In [232]:
n=0

# nth document's topic distribution
print(ldamodel.get_document_topics(corpus_[n]))

# nth document's category
#print(categories[n])

# show the original document
print(" ".join(docs[n]))

[(1, 0.30122235), (16, 0.6784536)]
The Fulton County Grand Jury said Friday an investigation of Atlanta 's recent primary election produced `` no evidence '' that any irregularities took place . The jury further said in term end presentments that the City Executive Committee , which had over-all charge of the election , `` deserves the praise and thanks of the City of Atlanta '' for the manner in which the election was conducted . The September October term jury had been charged by Fulton Superior Court Judge Durwood Pye to investigate reports of possible `` irregularities '' in the hard-fought primary which was won by Mayor-nominate Ivan Allen Jr. . `` Only a relative handful of such reports was received '' , the jury said , `` considering the widespread interest in the election , the number of voters and the size of this city '' . The jury said it did find that many of Georgia 's registration and election laws `` are outmoded or inadequate and often ambiguous '' . It recommended that

## Visualization
We can further analyze our result through visualization.

In [233]:
import pyLDAvis.gensim
pyLDAvis.enable_notebook()
lda_display = pyLDAvis.gensim.prepare(ldamodel, corpus_, dictionary)
lda_display

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
13     0.009900  0.012943       1        1  14.759728
16     0.005626  0.000585       2        1   9.444222
19     0.000127 -0.008900       3        1   9.239143
12     0.001724 -0.010689       4        1   8.495790
11     0.010767 -0.006538       5        1   8.463702
7      0.003705 -0.000396       6        1   8.219635
1      0.010816  0.004102       7        1   7.277559
17    -0.000775 -0.002927       8        1   5.982613
8      0.004303 -0.002415       9        1   5.813221
0      0.002546  0.001863      10        1   5.351235
3      0.000494 -0.000585      11        1   2.454369
15    -0.003867 -0.004142      12        1   2.433021
18    -0.002119  0.001622      13        1   1.957290
10    -0.004705  0.002712      14        1   1.851400
6     -0.001287  0.001030      15        1   1.732340
5     -0.008666 -0.001058      16        1   1.706134
2     -0.003176  0.002004      17        1   1.468742
4     -0.008746  0.001905      18        1   1.455836
14    -0.010334  0.002262      19        1   1.013017
9     -0.006335  0.006624      20        1   0.880988, topic_info=          Term         Freq        Total Category  logprob  loglift
459      state  1002.000000  1002.000000  Default  30.0000  30.0000
481       time   974.000000   974.000000  Default  29.0000  29.0000
284        man   823.000000   823.000000  Default  28.0000  28.0000
1604      form   334.000000   334.000000  Default  27.0000  27.0000
841     people   549.000000   549.000000  Default  26.0000  26.0000
...        ...          ...          ...      ...      ...      ...
1026  american     3.310909   446.623749  Topic20  -6.7080  -0.1726
471     system     2.951668   310.874329  Topic20  -6.8229   0.0749
3358      life     3.318295   495.329437  Topic20  -6.7058  -0.2739
78       child     2.976595   386.352203  Topic20  -6.8145  -0.1341
433     school     2.981330   458.924408  Topic20  -6.8129  -0.3046

[1587 rows x 6 columns], token_table=       Topic      Freq Term
term                       
989        1  0.060560   'd
989        2  0.088511   'd
989        3  0.116461   'd
989        4  0.130437   'd
989        5  0.079194   'd
...      ...       ...  ...
26174     10  0.042813  zen
26174     11  0.042813  zen
9740       1  0.234021   |a
9740       2  0.234021   |a
9740       9  0.234021   |a

[8820 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[14, 17, 20, 13, 12, 8, 2, 18, 9, 1, 4, 16, 19, 11, 7, 6, 3, 5, 15, 10])